In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
import joblib

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

In [2]:
from src.Bayesian_state import *
from src.Bayesian_state.problems.config import config_fgt
from src.Bayesian_state.problems import *
# from src.Bayesian_state.utils.optimizer import Optimizer, ConstantDict
# from src.Bayesian_state.utils.model_evaluation import ModelEval

INFO:cat-learning:logger is running normally.


d:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\logs\Run_20251009_172805.log
{'default_model': {'modules': {}, 'agenda': ['__self__']}, 'pmh_model': {'modules': {'memory': {'class': 'BaseMemory', 'kwargs': {}}, 'likelihood': {'class': 'BaseLikelihood', 'kwargs': {}}, 'perception': {'class': 'BasePerception', 'kwargs': {}}, 'hypotheses': {'class': 'BaseHypotheses', 'kwargs': {}}}, 'agenda': ['perception', 'likelihood', 'hypotheses', 'memory']}}


In [3]:
processed_path = Path(project_root) / 'data' / 'processed'
learning_data = pd.read_csv(processed_path / 'Task2_processed.csv')

subject_id = 1
subject_data = learning_data[learning_data["iSub"] == subject_id]
condition = subject_data["condition"].iloc[0]
s_data = (
    subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
    subject_data["choice"].values,
    subject_data["feedback"].values
)

In [4]:
beta_range = (1, 30)
beta_resolution = 5
beta_grid = np.linspace(beta_range[0], beta_range[1], beta_resolution)

In [5]:
from src.Bayesian_state.problems import StandardModel, StateModel
from src.Bayesian_state.utils.load_config import MODEL_STRUCT

# 打印字典并排版
import pprint
pprint.pprint(MODEL_STRUCT)

model = StateModel(MODEL_STRUCT['default_model'])

{'default_model': {'agenda': ['__self__'], 'modules': {}},
 'pmh_model': {'agenda': ['perception', 'likelihood', 'hypotheses', 'memory'],
               'modules': {'hypotheses': {'class': 'BaseHypotheses',
                                          'kwargs': {}},
                           'likelihood': {'class': 'BaseLikelihood',
                                          'kwargs': {}},
                           'memory': {'class': 'BaseMemory', 'kwargs': {}},
                           'perception': {'class': 'BasePerception',
                                          'kwargs': {}}}}}
All modules built successfully.


In [6]:
results_state = model.fit_step_by_step(s_data)

TypeError: '<=' not supported between instances of 'NoneType' and 'float'

In [7]:
results_prev = model.fit_step_by_step(s_data, beta_grid, slicing="all")

In [10]:
results_state

[{'best_k': 17,
  'best_beta': 15.5,
  'best_params': {'k': 17, 'beta': 15.5},
  'best_log_likelihood': None,
  'best_norm_posterior': 0.07408199037095974,
  'hypo_details': {0: {'beta_opt': 15.5,
    'll_max': None,
    'post_sum': 0.04043871457227801,
    'post_max': 0.008677487776920683,
    'is_best': False},
   1: {'beta_opt': 30.0,
    'll_max': None,
    'post_sum': 0.03775339211549131,
    'post_max': 0.008198055366647708,
    'is_best': False},
   2: {'beta_opt': 8.25,
    'll_max': None,
    'post_sum': 0.04142671237624679,
    'post_max': 0.009128207522982573,
    'is_best': False},
   3: {'beta_opt': 30.0,
    'll_max': None,
    'post_sum': 0.04588651333020696,
    'post_max': 0.013910107004606707,
    'is_best': False},
   4: {'beta_opt': 8.25,
    'll_max': None,
    'post_sum': 0.04956307183315816,
    'post_max': 0.014008975127750655,
    'is_best': False},
   5: {'beta_opt': 30.0,
    'll_max': None,
    'post_sum': 0.047329687572689225,
    'post_max': 0.012839092588